In [1]:
import healpy as hp
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from scipy import optimize


In [2]:
def mkdir(path):
    if not os.path.exists(path):
        os.makedirs(path)


In [5]:
NSIDE = 2**10
NPIX = hp.nside2npix(NSIDE)
reso = 1
drawdeg = 3
binsnumber = int(drawdeg * 60 / reso * 2)


In [7]:
def LIMA(alpha, Non, Noff):
    sig = np.sqrt(2) * np.sqrt(
        Non * np.log((1 + alpha) / alpha * (Non / (Non + Noff)))
        + Noff * np.log((1 + alpha) * Noff / (Non + Noff))
    )
    if type(sig) is np.ndarray:
        sig[np.where((Non - Noff * alpha) < 0)] = -sig[
            np.where((Non - Noff * alpha) < 0)
        ]
        sig[np.isnan(sig)] = 0

    else:
        if (Non - Noff * alpha) < 0:
            sig = -sig
        if np.isnan(sig):
            sig = 0
    return sig


In [8]:
DataPath = (
    "/home2/hky/github/Gamma_Energy/Exptdata/ALLsky_23_05_17_isgammacuted_E_Ra_Dec_new"
)
Exptdata = dict()
for root, dirs, files in os.walk(DataPath):
    for name in files:
        Exptdata_tmp = np.load(os.path.join(root, name))
        for key in Exptdata_tmp:
            if key not in Exptdata.keys():
                Exptdata[key] = list()
            Exptdata[key].append(Exptdata_tmp[key])
for key in Exptdata.keys():
    Exptdata[key] = np.concatenate(Exptdata[key])
Exptdata["sumpfcut"] = np.zeros_like(Exptdata["sumpf"])

In [ ]:
def getsigma_Allsky(cut, sumpfbins, Exptdata):
    Exptdata["sumpfcut"][Exptdata["sumpf"] < sumpfbins[0]] = 1
    Exptdata["sumpfcut"][Exptdata["sumpf"] > sumpfbins[-1]] = 0.99
    for i in range(len(cut)):
        Exptdata["sumpfcut"][
            (Exptdata["sumpf"] > sumpfbins[i]) & (Exptdata["sumpf"] < sumpfbins[i + 1])
        ] = cut[i]
    hpmap_All = np.zeros(NPIX)
    hpmap_Background = np.zeros(NPIX)
    need = np.where(Exptdata["isgamma"] > Exptdata["sumpfcut"])
    np.add.at(
        hpmap_All,
        hp.ang2pix(
            NSIDE,
            Exptdata["Ra"][need],
            Exptdata["Dec"][need],
            lonlat=True,
        ),
        1,
    )
    for i in range(20):
        np.add.at(
            hpmap_Background,
            hp.ang2pix(
                NSIDE,
                Exptdata[f"RaOff_{i}"][need],
                Exptdata[f"DecOff_{i}"][need],
                lonlat=True,
            ),
            1,
        )
    return hpmap_All, hpmap_Background


In [ ]:
sumpfbins = np.logspace(1.6, 4, 13)
# cut0 = np.array(
#     [0.11, 0.21, 0.21, 0.21, 0.66, 0.76, 0.96, 0.98, 0.99, 0.99, 0.99, 0.99]
# )
cut0 = np.full(len(sumpfbins)-1,1)
cut0[0]=0.11
smoothangle = 0.5

In [ ]:
cut = cut0
Exptdata["sumpfcut"][Exptdata["sumpf"] < sumpfbins[0]] = 1
Exptdata["sumpfcut"][Exptdata["sumpf"] > sumpfbins[-1]] = 0.99
for i in range(len(cut)):
    Exptdata["sumpfcut"][
        (Exptdata["sumpf"] > sumpfbins[i]) & (Exptdata["sumpf"] < sumpfbins[i + 1])
    ] = cut[i]
hpmap_All = np.zeros(NPIX)
hpmap_Background = np.zeros(NPIX)
need = np.where(Exptdata["isgamma"] > Exptdata["sumpfcut"])
np.add.at(
    hpmap_All,
    hp.ang2pix(
        NSIDE,
        Exptdata["Ra"][need],
        Exptdata["Dec"][need],
        lonlat=True,
    ),
    1,
)

In [ ]:
hpmap_All, hpmap_Background = getsigma_Allsky(cut0, sumpfbins, Exptdata)

In [ ]:
hpmap_All_smoothed = np.zeros(NPIX)
hpmap_Background_smoothed = np.zeros(NPIX)
for pix in range(NPIX):
    pixneed = hp.query_disc(NSIDE, hp.pix2vec(NSIDE, pix), np.radians(smoothangle))
    hpmap_All_smoothed[pix] = np.sum(hpmap_All[pixneed])
    hpmap_Background_smoothed[pix] = np.sum(hpmap_Background[pixneed])

In [ ]:
def top_hat(b, radius):
    return np.where(abs(b)<=radius, 1, 0)

b = np.linspace(0,np.pi,1000000)
bw = top_hat(b, np.radians(0.5)) #top_hat function of radius 45°
beam = hp.sphtfunc.beam2bl(bw, b, NSIDE*3)

hpmap_All_smoothed2 = hp.smoothing(hpmap_All, beam_window=beam)
hpmap_Background_smoothed2 = hp.smoothing(hpmap_Background, beam_window=beam)

In [ ]:
np.mean((hpmap_All_smoothed/hpmap_All_smoothed2)[hp.query_strip(NSIDE,0,np.pi/2)])

In [ ]:
hp.mollview(hpmap_All_smoothed/hpmap_All_smoothed2)

In [ ]:
hp.mollview(hpmap_All_smoothed-hpmap_Background_smoothed/20)

In [ ]:
def getExptOn(Exptdata, Ra, Dec, Energymin, Energymax):
    ExptOn_cuted = np.where(
        (Exptdata["Ra"] > Ra - 4 / np.cos(np.deg2rad(Dec)))
        & (Exptdata["Ra"] < Ra + 4 / np.cos(np.deg2rad(Dec)))
        & (Exptdata["Dec"] > Dec - 4)
        & (Exptdata["Dec"] < Dec + 4)
        & (Exptdata["energy"] > Energymin)
        & (Exptdata["energy"] < Energymax)
    )
    ExptOn_Ra = Exptdata["Ra"][ExptOn_cuted]
    ExptOn_Dec = Exptdata["Dec"][ExptOn_cuted]
    ExptOn_isgamma = Exptdata["isgamma"][ExptOn_cuted]
    ExptOn_sumpf = Exptdata["sumpf"][ExptOn_cuted]
    return ExptOn_Ra, ExptOn_Dec, ExptOn_isgamma, ExptOn_sumpf


def getExptOff(Exptdata, Ra, Dec, Energymin, Energymax):
    ExptOff_Ra = list()
    ExptOff_Dec = list()
    ExptOff_isgamma = list()
    ExptOff_sumpf = list()
    for i in range(20):
        ExptOff_cuted = np.where(
            (Exptdata[f"RaOff_{i}"] > Ra - 4 / np.cos(np.deg2rad(Dec)))
            & (Exptdata[f"RaOff_{i}"] < Ra + 4 / np.cos(np.deg2rad(Dec)))
            & (Exptdata[f"DecOff_{i}"] > Dec - 4)
            & (Exptdata[f"DecOff_{i}"] < Dec + 4)
            & (Exptdata["energy"] > Energymin)
            & (Exptdata["energy"] < Energymax)
        )
        ExptOff_Ra.append(Exptdata[f"RaOff_{i}"][ExptOff_cuted])
        ExptOff_Dec.append(Exptdata[f"DecOff_{i}"][ExptOff_cuted])
        ExptOff_isgamma.append(Exptdata["isgamma"][ExptOff_cuted])
        ExptOff_sumpf.append(Exptdata["sumpf"][ExptOff_cuted])
    ExptOff_Ra = np.concatenate(ExptOff_Ra)
    ExptOff_Dec = np.concatenate(ExptOff_Dec)
    ExptOff_isgamma = np.concatenate(ExptOff_isgamma)
    ExptOff_sumpf = np.concatenate(ExptOff_sumpf)
    return ExptOff_Ra, ExptOff_Dec, ExptOff_isgamma, ExptOff_sumpf


def getExptOn_Allsky(Exptdata, Energymin, Energymax):
    ExptOn_cuted = np.where(
        (Exptdata["energy"] > Energymin) & (Exptdata["energy"] < Energymax)
    )
    ExptOn_Ra = Exptdata["Ra"][ExptOn_cuted]
    ExptOn_Dec = Exptdata["Dec"][ExptOn_cuted]
    ExptOn_isgamma = Exptdata["isgamma"][ExptOn_cuted]
    ExptOn_sumpf = Exptdata["sumpf"][ExptOn_cuted]
    return ExptOn_Ra, ExptOn_Dec, ExptOn_isgamma, ExptOn_sumpf


def getExptOff_Allsky(Exptdata, Energymin, Energymax):
    ExptOff_Ra = list()
    ExptOff_Dec = list()
    ExptOff_isgamma = list()
    ExptOff_sumpf = list()
    for i in range(20):
        ExptOff_cuted = np.where(
            (Exptdata["energy"] > Energymin) & (Exptdata["energy"] < Energymax)
        )
        ExptOff_Ra.append(Exptdata[f"RaOff_{i}"][ExptOff_cuted])
        ExptOff_Dec.append(Exptdata[f"DecOff_{i}"][ExptOff_cuted])
        ExptOff_isgamma.append(Exptdata["isgamma"][ExptOff_cuted])
        ExptOff_sumpf.append(Exptdata["sumpf"][ExptOff_cuted])
    ExptOff_Ra = np.concatenate(ExptOff_Ra)
    ExptOff_Dec = np.concatenate(ExptOff_Dec)
    ExptOff_isgamma = np.concatenate(ExptOff_isgamma)
    ExptOff_sumpf = np.concatenate(ExptOff_sumpf)
    return ExptOff_Ra, ExptOff_Dec, ExptOff_isgamma, ExptOff_sumpf


def gethpmap_All_Background(
    cut,
    ExptOn_Ra,
    ExptOn_Dec,
    ExptOn_isgamma,
    ExptOn_sumpf,
    ExptOff_Ra,
    ExptOff_Dec,
    ExptOff_isgamma,
    ExptOff_sumpf,
    sumpfbins,
):
    hpmap_All = np.zeros(NPIX)
    hpmap_Background = np.zeros(NPIX)
    ExptOn_sumpfcut = np.zeros_like(ExptOn_isgamma)
    ExptOff_sumpfcut = np.zeros_like(ExptOff_isgamma)
    ExptOn_sumpfcut[ExptOn_sumpf < sumpfbins[0]] = 1
    ExptOn_sumpfcut[ExptOn_sumpf > sumpfbins[-1]] = 0.99
    ExptOff_sumpfcut[ExptOff_sumpf < sumpfbins[0]] = 1
    ExptOff_sumpfcut[ExptOff_sumpf > sumpfbins[-1]] = 0.99
    for i in range(len(cut)):
        ExptOn_sumpfcut[
            (ExptOn_sumpf < sumpfbins[i + 1]) & (ExptOn_sumpf > sumpfbins[i])
        ] = cut[i]
        ExptOff_sumpfcut[
            (ExptOff_sumpf < sumpfbins[i + 1]) & (ExptOff_sumpf > sumpfbins[i])
        ] = cut[i]
    np.add.at(
        hpmap_All,
        hp.ang2pix(
            NSIDE,
            ExptOn_Ra[ExptOn_isgamma > ExptOn_sumpfcut],
            ExptOn_Dec[ExptOn_isgamma > ExptOn_sumpfcut],
            lonlat=True,
        ),
        1,
    )
    np.add.at(
        hpmap_Background,
        hp.ang2pix(
            NSIDE,
            ExptOff_Ra[ExptOff_isgamma > ExptOff_sumpfcut],
            ExptOff_Dec[ExptOff_isgamma > ExptOff_sumpfcut],
            lonlat=True,
        ),
        1,
    )
    return hpmap_All, hpmap_Background


def getsigma(Ra, Dec, smoothangle, hpmap_All, hpmap_Background):
    hpmap_All_smoothed = np.zeros(NPIX)
    hpmap_Background_smoothed = np.zeros(NPIX)
    for pix in hp.query_disc(NSIDE, hp.ang2vec(Ra, Dec, lonlat=True), np.deg2rad(4)):
        pixneed = hp.query_disc(NSIDE, hp.pix2vec(NSIDE, pix), np.radians(smoothangle))
        hpmap_All_smoothed[pix] = np.sum(hpmap_All[pixneed])
        hpmap_Background_smoothed[pix] = np.sum(hpmap_Background[pixneed])
    sig = LIMA(0.05, hpmap_All_smoothed, hpmap_Background_smoothed)
    return sig


def getsigma_Allsky(smoothangle, hpmap_All, hpmap_Background):
    hpmap_All_smoothed = np.zeros(NPIX)
    hpmap_Background_smoothed = np.zeros(NPIX)
    for pix in range(NPIX):
        pixneed = hp.query_disc(NSIDE, hp.pix2vec(NSIDE, pix), np.radians(smoothangle))
        hpmap_All_smoothed[pix] = np.sum(hpmap_All[pixneed])
        hpmap_Background_smoothed[pix] = np.sum(hpmap_Background[pixneed])
    sig = LIMA(0.05, hpmap_All_smoothed, hpmap_Background_smoothed)
    return sig


def getsigma_foropt(
    cut,
    sumpfbins,
    Ra,
    Dec,
    smoothangle,
    ExptOn_Ra,
    ExptOn_Dec,
    ExptOn_isgamma,
    ExptOn_sumpf,
    ExptOff_Ra,
    ExptOff_Dec,
    ExptOff_isgamma,
    ExptOff_sumpf,
):
    hpmap_All, hpmap_Background = gethpmap_All_Background(
        cut,
        ExptOn_Ra,
        ExptOn_Dec,
        ExptOn_isgamma,
        ExptOn_sumpf,
        ExptOff_Ra,
        ExptOff_Dec,
        ExptOff_isgamma,
        ExptOff_sumpf,
        sumpfbins,
    )
    sig = getsigma(Ra, Dec, smoothangle, hpmap_All, hpmap_Background)
    # print(np.max(sig))
    return -np.max(sig)


def getsigma_fordraw(
    cut,
    sumpfbins,
    Ra,
    Dec,
    smoothangle,
    ExptOn_Ra,
    ExptOn_Dec,
    ExptOn_isgamma,
    ExptOn_sumpf,
    ExptOff_Ra,
    ExptOff_Dec,
    ExptOff_isgamma,
    ExptOff_sumpf,
):
    hpmap_All, hpmap_Background = gethpmap_All_Background(
        cut,
        ExptOn_Ra,
        ExptOn_Dec,
        ExptOn_isgamma,
        ExptOn_sumpf,
        ExptOff_Ra,
        ExptOff_Dec,
        ExptOff_isgamma,
        ExptOff_sumpf,
        sumpfbins,
    )
    sig = getsigma(Ra, Dec, smoothangle, hpmap_All, hpmap_Background)
    return sig


def getsigma_Allsky_fordraw(
    cut,
    sumpfbins,
    smoothangle,
    Exptdata,
    Energymin,
    Energymax,
):
    ExptOn_Ra, ExptOn_Dec, ExptOn_isgamma, ExptOn_sumpf = getExptOn_Allsky(
        cut, sumpfbins, Exptdata, Energymin, Energymax
    )
    ExptOff_Ra, ExptOff_Dec, ExptOff_isgamma, ExptOff_sumpf = getExptOff_Allsky(
        cut, sumpfbins, Exptdata, Energymin, Energymax
    )
    hpmap_All, hpmap_Background = gethpmap_All_Background(
        cut,
        ExptOn_Ra,
        ExptOn_Dec,
        ExptOn_isgamma,
        ExptOn_sumpf,
        ExptOff_Ra,
        ExptOff_Dec,
        ExptOff_isgamma,
        ExptOff_sumpf,
        sumpfbins,
    )
    sig = getsigma_Allsky(smoothangle, hpmap_All, hpmap_Background)
    return sig


sumpfbins = np.logspace(1.6, 4, 13)
# cut0 = np.linspace(0, 0.99, 9)
# cut0 = [ 0.9  ,0.96  ,0.95 , 0.73, 0.5, 0.45,  0.64,  0.6,  0.57]
cut0 = [0.11, 0.21, 0.21, 0.21, 0.66, 0.76, 0.96, 0.98, 0.99, 0.99, 0.99, 0.99]

bounds = [(0.2, 0.99) for _ in range(12)]


In [ ]:
TeVdata = pd.read_table("/home2/hky/github/Gamma_Energy/AllSky_withCR/TeVcat.log")
Ra_TeVcat = TeVdata["Ra"].to_numpy()
Dec_TeVcat = TeVdata["Dec"].to_numpy()
for i in range(len(Ra_TeVcat)):
    Ra_TeVcat_tmp = Ra_TeVcat[i].split()
    Dec_TeVcat_tmp = Dec_TeVcat[i].split()
    Ra_TeVcat[i] = (
        float(Ra_TeVcat_tmp[0]) / 24
        + float(Ra_TeVcat_tmp[1]) / 24 / 60
        + float(Ra_TeVcat_tmp[2]) / 24 / 60 / 60
    ) * 360
    Dec_TeVcat[i] = float(Dec_TeVcat_tmp[0])
    delta_Dec_TeVcat = (
        float(Dec_TeVcat_tmp[1]) / 60 + float(Dec_TeVcat_tmp[2]) / 60 / 60
    )
    Dec_TeVcat[i] += (-1) ** (Dec_TeVcat[i] < 0) * delta_Dec_TeVcat

Ra_TeVcat = Ra_TeVcat.astype(np.float32)
Dec_TeVcat = Dec_TeVcat.astype(np.float32)
TeVname = TeVdata["Name"]
TeVtype = TeVdata["Type"]


In [ ]:
Energymin = 10
Energymax = np.inf
sigma_max = 0
smoothangle = 0.5
cut0_tmp = np.array(cut0)
cut0_tmp[1:]=1
sigam_Allsky = getsigma_Allsky_fordraw(cut0_tmp,
    sumpfbins,
    smoothangle,
    Exptdata,
    Energymin,
    Energymax)

In [ ]:
Ra = 54.5
Dec = 53
Energymin = 10
Energymax = np.inf
sigma_max = 0
smoothangle = 0.5


In [ ]:

ExptOn_Ra, ExptOn_Dec, ExptOn_isgamma, ExptOn_sumpf = getExptOn(
    Exptdata, Ra, Dec, Energymin, Energymax
)
ExptOff_Ra, ExptOff_Dec, ExptOff_isgamma, ExptOff_sumpf = getExptOff(
    Exptdata, Ra, Dec, Energymin, Energymax
)


In [ ]:
getsigma_foropt(
    cut0,
    sumpfbins,
    Ra,
    Dec,
    smoothangle,
    ExptOn_Ra,
    ExptOn_Dec,
    ExptOn_isgamma,
    ExptOn_sumpf,
    ExptOff_Ra,
    ExptOff_Dec,
    ExptOff_isgamma,
    ExptOff_sumpf,
)


In [ ]:
res = optimize.differential_evolution(getsigma_foropt, bounds,args=(
        sumpfbins,
        Ra,
        Dec,
        smoothangle,
        ExptOn_Ra,
        ExptOn_Dec,
        ExptOn_isgamma,
        ExptOn_sumpf,
        ExptOff_Ra,
        ExptOff_Dec,
        ExptOff_isgamma,
        ExptOff_sumpf,
    ),)

In [ ]:
 [ 6.752e-01  6.921e-01  9.542e-01  8.447e-01  5.020e-01  8.840e-01  6.081e-01  9.398e-01  3.776e-01]
[ 8.975e-01  9.592e-01  9.475e-01  7.330e-01  4.949e-01   4.545e-01  6.411e-01  6.015e-01  5.695e-01]

In [ ]:
res

In [ ]:
sig = getsigma_fordraw(cut0,sumpfbins,
        Ra,
        Dec,
        smoothangle,
        ExptOn_Ra,
        ExptOn_Dec,
        ExptOn_isgamma,
        ExptOn_sumpf,
        ExptOff_Ra,
        ExptOff_Dec,
        ExptOff_isgamma,
        ExptOff_sumpf,)

In [ ]:
sig = getsigma_Allsky(smoothangle, hpmap_All, hpmap_Background)

In [ ]:
# sig_copy = sig
# sig_copy[sig_copy<3.5]=0
hp.mollview(sig)
hp.graticule()

In [ ]:
# Ra = 317
# Dec = 52
sigma_tmp = hp.gnomview(
    sig,
    rot=[Ra, Dec],
    xsize=drawdeg * 60 / reso * 2,
    reso=reso,
    return_projected_map=True,
    no_plot=True,
)
sigma_inverse = np.zeros_like(sigma_tmp)
for i in range(sigma_tmp.shape[0]):
    sigma_inverse[:, i] = sigma_tmp[:, sigma_tmp.shape[0] - 1 - i]
fig, ax = plt.subplots(figsize=(8, 6))
c = ax.pcolormesh(
    np.linspace(
        Ra - drawdeg / np.cos(np.deg2rad(Dec)),
        Ra + drawdeg / np.cos(np.deg2rad(Dec)),
        binsnumber,
    ),
    np.linspace(
        Dec - drawdeg,
        Dec + drawdeg,
        binsnumber,
    ),
    sigma_inverse,
    cmap="plasma",
    vmin=0,
)
Ra_min = Ra - drawdeg / np.cos(np.deg2rad(Dec))
Ra_max = Ra + drawdeg / np.cos(np.deg2rad(Dec))
Dec_min = Dec - drawdeg
Dec_max = Dec + drawdeg
fig.colorbar(c, orientation="vertical")
ax.set_xlim(
    Ra - drawdeg / np.cos(np.deg2rad(Dec)),
    Ra + drawdeg / np.cos(np.deg2rad(Dec)),
)
ax.set_ylim(Dec - drawdeg, Dec + drawdeg)
ax.invert_xaxis()

In [ ]:
(21/24+39/24/60+23.5/24/60/60)*360

In [ ]:
res = optimize.minimize(
    getsigma_foropt,
    cut0,
    method="Nelder-Mead",
    args=(
        sumpfbins,
        Ra,
        Dec,
        smoothangle,
        ExptOn_Ra,
        ExptOn_Dec,
        ExptOn_isgamma,
        ExptOn_sumpf,
        ExptOff_Ra,
        ExptOff_Dec,
        ExptOff_isgamma,
        ExptOff_sumpf,
    ),
    options={"disp": True, "adaptive": True},
)


In [ ]:
res

In [ ]:
getsigma_foropt(
    res.x,
    sumpfbins,
    Ra,
    Dec,
    smoothangle,
    ExptOn_Ra,
    ExptOn_Dec,
    ExptOn_isgamma,
    ExptOn_sumpf,
    ExptOff_Ra,
    ExptOff_Dec,
    ExptOff_isgamma,
    ExptOff_sumpf,
)


In [ ]:
results = dict()
results["shgo"] = optimize.shgo(
    getsigma_foropt,
    bounds,
    args=(
        sumpfbins,
        Ra,
        Dec,
        smoothangle,
        ExptOn_Ra,
        ExptOn_Dec,
        ExptOn_isgamma,
        ExptOn_sumpf,
        ExptOff_Ra,
        ExptOff_Dec,
        ExptOff_isgamma,
        ExptOff_sumpf,
    ),
)


In [ ]:
Ra = 83.6
Dec = 22
Energymin = 10
Energymax = np.inf
sigma_max = 0

ExptOn_Ra, ExptOn_Dec, ExptOn_isgamma, ExptOn_sumpf = getExptOn(
    Exptdata, Ra, Dec, Energymin, Energymax
)
ExptOff_Ra, ExptOff_Dec, ExptOff_isgamma, ExptOff_sumpf = getExptOff(
    Exptdata, Ra, Dec, Energymin, Energymax
)
sumpfbins = np.logspace(1.6, 4, 25)
cut = np.linspace(0, 0.99, 24)
hpmap_All, hpmap_Background = gethpmap_All_Background(
    cut,
    ExptOn_Ra,
    ExptOn_Dec,
    ExptOn_isgamma,
    ExptOn_sumpf,
    ExptOff_Ra,
    ExptOff_Dec,
    ExptOff_isgamma,
    ExptOff_sumpf,
    sumpfbins,
)
smoothangle = 0.5
print(getsigma(Ra, Dec, smoothangle, hpmap_All, hpmap_Background))


In [ ]:
savepath = "/home2/hky/github/Gamma_Energy/AllSky_withCR/Exptdatacut/fig"
Energymin_list = [10, 10, 50, 50, 80, 100]
Energymax_list = [np.inf, 50, np.inf, 100, np.inf, np.inf]
for Tname, Ttype, Ra, Dec in zip(TeVname, TeVtype, Ra_TeVcat, Dec_TeVcat):
    for Energymin, Energymax in zip(Energymin_list, Energymax_list):
        sigma_max = 0
        
        ExptOn_Ra,ExptOn_Dec,ExptOn_isgamma,ExptOn_sumpf=getExptOn(Exptdata,Ra,Dec,Energymin,Energymax)
        ExptOff_Ra,ExptOff_Dec,ExptOff_isgamma,ExptOff_sumpf  =getExptOff(Exptdata,Ra,Dec,Energymin,Energymax)
       

        for cut in np.linspace(0.01, 0.99, 99):
            hpmap_All, hpmap_Background=gethpmap_All_Background(ExptOn_Ra,ExptOn_Dec,ExptOn_isgamma,ExptOff_Ra,ExptOff_Dec,ExptOff_isgamma,cut)
            for smoothangle in [0.3, 0.5, 0.8, 1, 1.5, 2, 3]:


        # if sigma_max > 3:
        #     hpmap_All = np.zeros(NPIX)
        #     hpmap_Background = np.zeros(NPIX)
        #     np.add.at(
        #         hpmap_All,
        #         hp.ang2pix(
        #             NSIDE,
        #             ExptOn_Ra[ExptOn_isgamma > cut_best],
        #             ExptOn_Dec[ExptOn_isgamma > cut_best],
        #             lonlat=True,
        #         ),
        #         1,
        #     )
        #     np.add.at(
        #         hpmap_Background,
        #         hp.ang2pix(
        #             NSIDE,
        #             ExptOff_Ra[ExptOff_isgamma > cut_best],
        #             ExptOff_Dec[ExptOff_isgamma > cut_best],
        #             lonlat=True,
        #         ),
        #         1,
        #     )
        #     hpmap_All_smoothed = np.zeros(NPIX)
        #     hpmap_Background_smoothed = np.zeros(NPIX)
        #     for pix in hp.query_disc(
        #         NSIDE, hp.ang2vec(Ra, Dec, lonlat=True), np.deg2rad(5)
        #     ):
        #         pixneed = hp.query_disc(
        #             NSIDE, hp.pix2vec(NSIDE, pix), np.radians(smoothanglebest)
        #         )
        #         hpmap_All_smoothed[pix] = np.sum(hpmap_All[pixneed])
        #         hpmap_Background_smoothed[pix] = np.sum(hpmap_Background[pixneed])
        #     sig = LIMA(0.05, hpmap_All_smoothed, hpmap_Background_smoothed)

        #     sigma_tmp = hp.gnomview(
        #         sig,
        #         rot=[Ra, Dec],
        #         xsize=drawdeg * 60 / reso * 2,
        #         reso=reso,
        #         return_projected_map=True,
        #         no_plot=True,
        #     )
        #     sigma_inverse = np.zeros_like(sigma_tmp)
        #     for i in range(sigma_tmp.shape[0]):
        #         sigma_inverse[:, i] = sigma_tmp[:, sigma_tmp.shape[0] - 1 - i]
        #     fig, ax = plt.subplots(figsize=(16, 9))
        #     c = ax.pcolormesh(
        #         np.linspace(
        #             Ra - drawdeg / np.cos(np.deg2rad(Dec)),
        #             Ra + drawdeg / np.cos(np.deg2rad(Dec)),
        #             binsnumber,
        #         ),
        #         np.linspace(
        #             Dec - drawdeg,
        #             Dec + drawdeg,
        #             binsnumber,
        #         ),
        #         sigma_inverse,
        #         cmap="plasma",
        #         vmin=0,
        #     )
        #     Ra_min = Ra - drawdeg / np.cos(np.deg2rad(Dec))
        #     Ra_max = Ra + drawdeg / np.cos(np.deg2rad(Dec))
        #     Dec_min = Dec - drawdeg
        #     Dec_max = Dec + drawdeg
        #     fig.colorbar(c, orientation="vertical")
        #     ax.set_xlim(
        #         Ra - drawdeg / np.cos(np.deg2rad(Dec)),
        #         Ra + drawdeg / np.cos(np.deg2rad(Dec)),
        #     )
        #     ax.set_ylim(Dec - drawdeg, Dec + drawdeg)
        #     ax.invert_xaxis()
        #     plt.show()
        #     break


In [ ]:
Ra = 284.5
Dec = 2
Energymin = 20
Energymax = np.inf
Exptdata_Crab = Exptdata["isgamma"][
    np.where(
        (Exptdata["Ra"] > Ra - 0.5)
        & (Exptdata["Ra"] < Ra + 0.5)
        & (Exptdata["Dec"] > Dec - 0.5)
        & (Exptdata["Dec"] < Dec + 0.5)
        & (Exptdata["energy"] > Energymin)
        & (Exptdata["energy"] < Energymax)
    )
]
Exptdata_Crab_background = list()
for i in range(20):
    Exptdata_Crab_background.append(
        Exptdata["isgamma"][
            np.where(
                (Exptdata[f"RaOff_{i}"] > Ra - 0.5)
                & (Exptdata[f"RaOff_{i}"] < Ra + 0.5)
                & (Exptdata[f"DecOff_{i}"] > Dec - 0.5)
                & (Exptdata[f"DecOff_{i}"] < Dec + 0.5)
                & (Exptdata["energy"] > Energymin)
                & (Exptdata["energy"] < Energymax)
            )
        ]
    )
Exptdata_Crab_background = np.concatenate(Exptdata_Crab_background)


In [ ]:
for cut in np.linspace(0.01, 0.99, 99):
    print(
        cut,
        LIMA(
            0.05,
            np.where(Exptdata_Crab > cut)[0].shape[0],
            np.where(Exptdata_Crab_background > cut)[0].shape[0],
        ),
    )


In [ ]:
energymin = 0
smoothangle = 0.5
data_need = np.where(
    (Exptdata["energy"] > energymin) & (Exptdata["isgamma"] > Exptdata["isgammacut"])
)
hpmap_All = np.zeros(NPIX)
hpmap_Background = np.zeros(NPIX)
hpmap_All_smoothed = np.zeros(NPIX)
hpmap_Background_smoothed = np.zeros(NPIX)
np.add.at(
    hpmap_All,
    hp.ang2pix(
        NSIDE,
        Exptdata["Ra"][data_need],
        Exptdata["Dec"][data_need],
        lonlat=True,
    ),
    1,
)
for i in range(20):
    np.add.at(
        hpmap_Background,
        hp.ang2pix(
            NSIDE,
            Exptdata[f"RaOff_{i}"][data_need],
            Exptdata[f"DecOff_{i}"][data_need],
            lonlat=True,
        ),
        1,
    )

# fwhm = np.deg2rad(smoothangle)
# hpmap_All_smoothed = hp.smoothing(hpmap_All, fwhm=fwhm)
# hpmap_Background_smoothed = hp.smoothing(hpmap_Background, fwhm=fwhm)

# sigma_hp = LIMA(0.05, hpmap_All_smoothed, hpmap_Background_smoothed)
# std = np.std(sigma_hp[(hpmap_All != 0)])
# std = np.std(sigma_hp[(hpmap_All != 0) & (np.abs(sigma_hp) < 5 * std)])
# sigma_hp /= std
for pix in range(NPIX):
    pixneed = hp.query_disc(NSIDE, hp.pix2vec(NSIDE, pix), np.radians(smoothangle))
    hpmap_All_smoothed[pix] = np.sum(hpmap_All[pixneed])
    hpmap_Background_smoothed[pix] = np.sum(hpmap_Background[pixneed])


In [ ]:
hp.mollview(hpmap_All)


In [ ]:
hp.mollview(hpmap_All_smoothed - hpmap_Background_smoothed / 20)


In [ ]:
plt.hist(sig[hpmap_All != 0], bins=100)
plt.yscale("log")
plt.show()


In [ ]:
hp.mollview(sig)


In [ ]:
np.max(sig)


In [ ]:
reso = 3
drawdeg = 3
binsnumber = int(drawdeg * 60 / reso * 2)
Ra = 284
Dec = 2
sigma_tmp = hp.gnomview(
    sig,
    rot=[Ra, Dec],
    xsize=drawdeg * 60 / reso * 2,
    reso=reso,
    return_projected_map=True,
    no_plot=True,
)
sigma_inverse = np.zeros_like(sigma_tmp)
for i in range(sigma_tmp.shape[0]):
    sigma_inverse[:, i] = sigma_tmp[:, sigma_tmp.shape[0] - 1 - i]
fig, ax = plt.subplots(figsize=(16, 9))
c = ax.pcolormesh(
    np.linspace(
        Ra - drawdeg / np.cos(np.deg2rad(Dec)),
        Ra + drawdeg / np.cos(np.deg2rad(Dec)),
        binsnumber,
    ),
    np.linspace(
        Dec - drawdeg,
        Dec + drawdeg,
        binsnumber,
    ),
    sigma_inverse,
    cmap="plasma",
    vmin=0,
)
Ra_min = Ra - drawdeg / np.cos(np.deg2rad(Dec))
Ra_max = Ra + drawdeg / np.cos(np.deg2rad(Dec))
Dec_min = Dec - drawdeg
Dec_max = Dec + drawdeg
fig.colorbar(c, orientation="vertical")
ax.set_xlim(
    Ra - drawdeg / np.cos(np.deg2rad(Dec)),
    Ra + drawdeg / np.cos(np.deg2rad(Dec)),
)
ax.set_ylim(Dec - drawdeg, Dec + drawdeg)
ax.invert_xaxis()
